# Migrating off ConversationTokenBufferMemory

Follow this guide if you're trying to migrate off one of the old memory classes listed below:


| Memory Type                      | Description                                                                                                                                                       |
|----------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `ConversationTokenBufferMemory`  | Keeps only the most recent messages in the conversation under the constraint that the total number of tokens in the conversation does not exceed a certain limit. |

`ConversationTokenBufferMemory` applies additional processing on top of the raw conversation history to trim the conversation history to a size that fits inside the context window of a chat model. 

This processing functionality can be accomplished using LangChain's built-in [trimMessages](https://api.js.langchain.com/functions/_langchain_core.messages.trimMessages.html) function.

```{=mdx}
:::important

We’ll begin by exploring a straightforward method that involves applying processing logic to the entire conversation history.

While this approach is easy to implement, it has a downside: as the conversation grows, so does the latency, since the logic is re-applied to all previous exchanges in the conversation at each turn.

More advanced strategies focus on incrementally updating the conversation history to avoid redundant processing.

For instance, the LangGraph [how-to guide on summarization](https://langchain-ai.github.io/langgraphjs/how-tos/add-summary-conversation-history/) demonstrates
how to maintain a running summary of the conversation while discarding older messages, ensuring they aren't re-processed during later turns.
:::
```

## Set up

### Dependencies

```{=mdx}
import Npm2Yarn from "@theme/Npm2Yarn"

<Npm2Yarn>
  @langchain/openai @langchain/core zod
</Npm2Yarn>
```

### Environment variables

```typescript
process.env.OPENAI_API_KEY = "YOUR_OPENAI_API_KEY";
```

## Reimplementing ConversationTokenBufferMemory logic

Here, we'll use `trimMessages` to keeps the system message and the most recent messages in the conversation under the constraint that the total number of tokens in the conversation does not exceed a certain limit.

<details open> 

In [1]:
import {
  AIMessage,
  HumanMessage,
  SystemMessage,
} from "@langchain/core/messages";

const messages = [
  new SystemMessage("you're a good assistant, you always respond with a joke."),
  new HumanMessage("i wonder why it's called langchain"),
  new AIMessage(
    'Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'
  ),
  new HumanMessage("and who is harrison chasing anyways"),
  new AIMessage(
      "Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!"
  ),
  new HumanMessage("why is 42 always the answer?"),
  new AIMessage(
      "Because it's the only number that's constantly right, even when it doesn't add up!"
  ),
  new HumanMessage("What did the cow say?"),
]

In [4]:
import { trimMessages } from "@langchain/core/messages";
import { ChatOpenAI } from "@langchain/openai";

const selectedMessages = await trimMessages(
  messages,
  {
    // Please see API reference for trimMessages for other ways to specify a token counter.
    tokenCounter: new ChatOpenAI({ model: "gpt-4o" }),
    maxTokens: 80,  // <-- token limit
    // The startOn is specified
    // to make sure we do not generate a sequence where
    // a ToolMessage that contains the result of a tool invocation
    // appears before the AIMessage that requested a tool invocation
    // as this will cause some chat models to raise an error.
    startOn: ["human", "ai"],
    strategy: "last",
    includeSystem: true,  // <-- Keep the system message
  }
)

for (const msg of selectedMessages) {
    console.log(msg);
}

SystemMessage {
  "content": "you're a good assistant, you always respond with a joke.",
  "additional_kwargs": {},
  "response_metadata": {}
}
HumanMessage {
  "content": "and who is harrison chasing anyways",
  "additional_kwargs": {},
  "response_metadata": {}
}
AIMessage {
  "content": "Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!",
  "additional_kwargs": {},
  "response_metadata": {},
  "tool_calls": [],
  "invalid_tool_calls": []
}
HumanMessage {
  "content": "why is 42 always the answer?",
  "additional_kwargs": {},
  "response_metadata": {}
}
AIMessage {
  "content": "Because it's the only number that's constantly right, even when it doesn't add up!",
  "additional_kwargs": {},
  "response_metadata": {},
  "tool_calls": [],
  "invalid_tool_calls": []
}
HumanMessage {
  "content": "What did the cow say?",
  "additional_kwargs": {},
  "response_metadata": {}
}


</details>

## Modern usage with LangGraph

The example below shows how to use LangGraph to add simple conversation pre-processing logic.

```{=mdx}
:::note

If you want to avoid running the computation on the entire conversation history each time, you can follow
the [how-to guide on summarization](https://langchain-ai.github.io/langgraphjs/how-tos/add-summary-conversation-history/) that demonstrates
how to discard older messages, ensuring they aren't re-processed during later turns.

:::
```

<details open>

In [15]:
import { v4 as uuidv4 } from 'uuid';
import { ChatOpenAI } from "@langchain/openai";
import { StateGraph, MessagesAnnotation, END, START, MemorySaver } from "@langchain/langgraph";
import { trimMessages } from "@langchain/core/messages";

// Define a chat model
const model = new ChatOpenAI({ model: "gpt-4o" });

// Define the function that calls the model
const callModel = async (state: typeof MessagesAnnotation.State): Promise<Partial<typeof MessagesAnnotation.State>> => {
  // highlight-start
  const selectedMessages = await trimMessages(
    state.messages,
    {
      tokenCounter: (messages) => messages.length, // Simple message count instead of token count
      maxTokens: 5, // Allow up to 5 messages
      strategy: "last",
      startOn: ["human", "ai"],
      includeSystem: true,
      allowPartial: false,
    }
  );
  // highlight-end

  const response = await model.invoke(selectedMessages);

  // With LangGraph, we're able to return a single message, and LangGraph will concatenate
  // it to the existing list
  return { messages: [response] };
};


// Define a new graph
const workflow = new StateGraph(MessagesAnnotation)
// Define the two nodes we will cycle between
  .addNode("model", callModel)
  .addEdge(START, "model")
  .addEdge("model", END)

const app = workflow.compile({
  // Adding memory is straightforward in LangGraph!
  // Just pass a checkpointer to the compile method.
  checkpointer: new MemorySaver()
});

// The thread id is a unique key that identifies this particular conversation
// ---
// NOTE: this must be `thread_id` and not `threadId` as the LangGraph internals expect `thread_id`
// ---
const thread_id = uuidv4();
const config = { configurable: { thread_id } };

const inputMessage = {
  role: "user",
  content: "hi! I'm bob",
}
for await (const event of await app.stream({ messages: [inputMessage] }, config)) {
  // LangGraph will stream events back inside objects where the key is the name of the node.
  const key = Object.keys(event)[0];
  if (key === "model") {
    const lastMessage = event[key].messages[event[key].messages.length - 1];
    console.log(lastMessage.content);
  }
}

// Here, let's confirm that the AI remembers our name!
const followUpMessage = {
  role: "user",
  content: "what was my name?",
}

// ---
// NOTE: You must pass the same thread id to continue the conversation
// we do that here by passing the same `config` object to the `.stream` call.
// ---
for await (const event of await app.stream({ messages: [followUpMessage] }, config)) {
  const key = Object.keys(event)[0];
  if (key === "model") {
    const lastMessage = event[key].messages[event[key].messages.length - 1];
    console.log(lastMessage.content);
  }
}

Hi Bob! How can I help you today?
Your name is Bob. How can I assist you further?


</details>

## Usage with a pre-built langgraph agent

This example shows usage of an Agent Executor with a pre-built agent constructed using the [create_tool_calling_agent](https://api.python.langchain.com/en/latest/agents/langchain.agents.tool_calling_agent.base.create_tool_calling_agent.html) function.

If you are using one of the [old LangChain pre-built agents](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/), you should be able
to replace that code with the new [langgraph pre-built agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/) which leverages
native tool calling capabilities of chat models and will likely work better out of the box.

<details open>

In [8]:
import uuid

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    trim_messages,
)
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent


@tool
def get_user_age(name: str) -> str:
    """Use this tool to find the user's age."""
    # This is a placeholder for the actual implementation
    if "bob" in name.lower():
        return "42 years old"
    return "41 years old"


memory = MemorySaver()
model = ChatOpenAI()


# highlight-start
def state_modifier(state) -> list[BaseMessage]:
    """Given the agent state, return a list of messages for the chat model."""
    # We're using the message processor defined above.
    return trim_messages(
        state["messages"],
        token_counter=len,  # <-- len will simply count the number of messages rather than tokens
        max_tokens=5,  # <-- allow up to 5 messages.
        strategy="last",
        # The start_on is specified
        # to make sure we do not generate a sequence where
        # a ToolMessage that contains the result of a tool invocation
        # appears before the AIMessage that requested a tool invocation
        # as this will cause some chat models to raise an error.
        start_on=("human", "ai"),
        include_system=True,  # <-- Keep the system message
        allow_partial=False,
    )


# highlight-end

app = create_react_agent(
    model,
    tools=[get_user_age],
    checkpointer=memory,
    # highlight-next-line
    state_modifier=state_modifier,
)

# The thread id is a unique key that identifies
# this particular conversation.
# We'll just generate a random uuid here.
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

# Tell the AI that our name is Bob, and ask it to use a tool to confirm
# that it's capable of working like an agent.
input_message = HumanMessage(content="hi! I'm bob. What is my age?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Confirm that the chat bot has access to previous conversation
# and can respond to the user saying that the user's name is Bob.
input_message = HumanMessage(content="do you remember my name?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob. What is my age?
================================== Ai Message ==================================
Tool Calls:
  get_user_age (call_jsMvoIFv970DhqqLCJDzPKsp)
 Call ID: call_jsMvoIFv970DhqqLCJDzPKsp
  Args:
    name: bob
================================= Tool Message =================================
Name: get_user_age

42 years old
================================== Ai Message ==================================

Bob, you are 42 years old.
================================ Human Message =================================

do you remember my name?
================================== Ai Message ==================================

Yes, your name is Bob.


In [1]:
import { z } from "zod";
import { v4 as uuidv4 } from 'uuid';
import { BaseMessage, trimMessages } from "@langchain/core/messages";
import { tool } from "@langchain/core/tools";
import { ChatOpenAI } from "@langchain/openai";
import { MemorySaver } from "@langchain/langgraph";
import { createReactAgent } from "@langchain/langgraph/prebuilt";

const getUserAge = tool(
  (name: string): string => {
    // This is a placeholder for the actual implementation
    if (name.toLowerCase().includes("bob")) {
      return "42 years old";
    }
    return "41 years old";
  },
  {
    name: "get_user_age",
    description: "Use this tool to find the user's age.",
    schema: z.string().describe("the name of the user"),
  }
);

const memory = new MemorySaver();
const model = new ChatOpenAI({ model: "gpt-4o" });

// highlight-start
const stateModifier = async (messages: BaseMessage[]): Promise<BaseMessage[]> => {
  // We're using the message processor defined above.
  return trimMessages(
    messages,
    {
      tokenCounter: (msgs) => msgs.length, // <-- .length will simply count the number of messages rather than tokens
      maxTokens: 5, // <-- allow up to 5 messages.
      strategy: "last",
      // The startOn is specified
      // to make sure we do not generate a sequence where
      // a ToolMessage that contains the result of a tool invocation
      // appears before the AIMessage that requested a tool invocation
      // as this will cause some chat models to raise an error.
      startOn: ["human", "ai"],
      includeSystem: true, // <-- Keep the system message
      allowPartial: false,
    }
  );
};
// highlight-end

const app = createReactAgent({
  llm: model,
  tools: [getUserAge],
  checkpointSaver: memory,
  // highlight-next-line
  messageModifier: async (msgs) => {
    console.log("Passed to msgModifier", msgs);
    const modified = await stateModifier(msgs);
    console.log("Returned from msgModifier", modified);
    return modified;
  },
});

// The thread id is a unique key that identifies
// this particular conversation.
// We'll just generate a random uuid here.
const threadId = uuidv4();
const config = { configurable: { thread_id: threadId } };

// Tell the AI that our name is Bob, and ask it to use a tool to confirm
// that it's capable of working like an agent.
const inputMessage = {
  role: "user",
  content: "hi! I'm bob. What is my age?",
}

for await (const event of await app.stream({ messages: [inputMessage] }, config)) {
  const key = Object.keys(event)[0];
  if (event[key] && "messages" in event[key]) {
    const lastMessage = event[key].messages[event[key].messages.length - 1];
    // console.log(lastMessage.content);
  }
}

// Confirm that the chat bot has access to previous conversation
// and can respond to the user saying that the user's name is Bob.
const followUpMessage = {
  role: "user",
  content: "do you remember my name?",
};

for await (const event of await app.stream({ messages: [followUpMessage] }, config)) {
  const key = Object.keys(event)[0];
  if (event[key] && "messages" in event[key]) {
    const lastMessage = event[key].messages[event[key].messages.length - 1];
    // console.log(lastMessage.content);
  }
}

Passed to msgModifier [
  HumanMessage {
    "id": "2f1d9e4a-043c-4ec2-9565-97bb939090c0",
    "content": "hi! I'm bob. What is my age?",
    "additional_kwargs": {},
    "response_metadata": {}
  }
]
Returned from msgModifier [
  HumanMessage {
    "id": "2f1d9e4a-043c-4ec2-9565-97bb939090c0",
    "content": "hi! I'm bob. What is my age?",
    "additional_kwargs": {},
    "response_metadata": {}
  }
]
Passed to msgModifier [
  HumanMessage {
    "id": "2f1d9e4a-043c-4ec2-9565-97bb939090c0",
    "content": "hi! I'm bob. What is my age?",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "id": "chatcmpl-AB6JpfxFiYluLN33MKNgMIh4pSHQ1",
    "content": "",
    "additional_kwargs": {
      "tool_calls": [
        {
          "id": "call_KAispSTkEY5VC4apLZK4vQtB",
          "type": "function",
          "function": "[Object]"
        }
      ]
    },
    "response_metadata": {
      "tokenUsage": {
        "completionTokens": 15,
        "promptTokens": 57,
    

BadRequestError: 400 An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_KAispSTkEY5VC4apLZK4vQtB
    at APIError.generate (/Users/bracesproul/code/lang-chain-ai/langchainjs/libs/langchain-openai/node_modules/openai/error.js:45:20)
    at OpenAI.makeStatusError (/Users/bracesproul/code/lang-chain-ai/langchainjs/libs/langchain-openai/node_modules/openai/core.js:275:33)
    at OpenAI.makeRequest (/Users/bracesproul/code/lang-chain-ai/langchainjs/libs/langchain-openai/node_modules/openai/core.js:318:30)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async /Users/bracesproul/code/lang-chain-ai/langchainjs/libs/langchain-openai/dist/chat_models.cjs:1362:29
    at async RetryOperation._fn (/Users/bracesproul/code/lang-chain-ai/langchainjs/node_modules/p-retry/index.js:50:12) {
  status: 400,
  headers: {
    'access-control-expose-

</details>

## LCEL: Add a preprocessing step

The simplest way to add complex conversation management is by introducing a pre-processing step in front of the chat model and pass the full conversation history to the pre-processing step.

This approach is conceptually simple and will work in many situations; for example, if using a [RunnableWithMessageHistory](/docs/how_to/message_history/) instead of wrapping the chat model, wrap the chat model with the pre-processor.

The obvious downside of this approach is that latency starts to increase as the conversation history grows because of two reasons:

1. As the conversation gets longer, more data may need to be fetched from whatever store your'e using to store the conversation history (if not storing it in memory).
2. The pre-processing logic will end up doing a lot of redundant computation, repeating computation from previous steps of the conversation.

:::caution

If you want to use a chat model's tool calling capabilities, remember to bind the tools to the model before adding the history pre-processing step to it!

:::

<details open>

In [9]:
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    trim_messages,
)
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

model = ChatOpenAI()


@tool
def what_did_the_cow_say() -> str:
    """Check to see what the cow said."""
    return "foo"


# highlight-start
message_processor = trim_messages(  # Returns a Runnable if no messages are provided
    token_counter=len,  # <-- len will simply count the number of messages rather than tokens
    max_tokens=5,  # <-- allow up to 5 messages.
    strategy="last",
    # The start_on is specified
    # to make sure we do not generate a sequence where
    # a ToolMessage that contains the result of a tool invocation
    # appears before the AIMessage that requested a tool invocation
    # as this will cause some chat models to raise an error.
    start_on=("human", "ai"),
    include_system=True,  # <-- Keep the system message
    allow_partial=False,
)
# highlight-end

# Note that we bind tools to the model first!
model_with_tools = model.bind_tools([what_did_the_cow_say])

# highlight-next-line
model_with_preprocessor = message_processor | model_with_tools

full_history = [
    SystemMessage("you're a good assistant, you always respond with a joke."),
    HumanMessage("i wonder why it's called langchain"),
    AIMessage(
        'Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'
    ),
    HumanMessage("and who is harrison chasing anyways"),
    AIMessage(
        "Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!"
    ),
    HumanMessage("why is 42 always the answer?"),
    AIMessage(
        "Because it’s the only number that’s constantly right, even when it doesn’t add up!"
    ),
    HumanMessage("What did the cow say?"),
]


# We pass it explicity to the model_with_preprocesor for illustrative purposes.
# If you're using `RunnableWithMessageHistory` the history will be automatically
# read from the source the you configure.
model_with_preprocessor.invoke(full_history).pretty_print()

================================== Ai Message ==================================
Tool Calls:
  what_did_the_cow_say (call_urHTB5CShhcKz37QiVzNBlIS)
 Call ID: call_urHTB5CShhcKz37QiVzNBlIS
  Args:


</details>

If you need to implement more efficient logic and want to use `RunnableWithMessageHistory` for now the way to achieve this
is to subclass from [BaseChatMessageHistory](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html) and
define appropriate logic for `add_messages` (that doesn't simply append the history, but instead re-writes it).

Unless you have a good reason to implement this solution, you should instead use LangGraph.

## Next steps

Explore persistence with LangGraph:

* [LangGraph quickstart tutorial](https://langchain-ai.github.io/langgraph/tutorials/introduction/)
* [How to add persistence ("memory") to your graph](https://langchain-ai.github.io/langgraph/how-tos/persistence/)
* [How to manage conversation history](https://langchain-ai.github.io/langgraph/how-tos/memory/manage-conversation-history/)
* [How to add summary of the conversation history](https://langchain-ai.github.io/langgraph/how-tos/memory/add-summary-conversation-history/)

Add persistence with simple LCEL (favor langgraph for more complex use cases):

* [How to add message history](/docs/how_to/message_history/)

Working with message history:

* [How to trim messages](/docs/how_to/trim_messages)
* [How to filter messages](/docs/how_to/filter_messages/)
* [How to merge message runs](/docs/how_to/merge_message_runs/)
